# Data Manipulation and Visualization - **Lifestyle**

progetto di: Tommaso Ciampolini

Questo notebook è stato sviluppato identificando come ipotetico cliente il **Governo Italiano**.

L'obiettivo di questa analisi è quello di mostrare l'indice di benessere dell'Italia rispetto agli altri Paesi, osservando il suo andamento negli ultimi anni e analizzando possibili fattori ad esso correlati con lo scopo di aiutare il Governo a prendere decisioni riguardanti le questioni pubbliche.

# Discovery

***

La seguente analisi indaga **la condizione di benessere dei cittadini italiani** in relazione alla situazione globale e mostra i diversi **parametri correlati** al benessere della persona. L'obiettivo consiste nel dare un'indicazione su quali aspetti focalizzare l'attenzione nell'ottica di **incrementare lo stato di benessere dei cittadini.**

L'analisi presenta la seguente struttura logica:

- Selezione e descrizione dei dati
- Pulizia ed esplorazione dei dati scelti
- Preparazione dei dati per la rappresentazione
- Panoramica generale sullo stato di benessere globale, negli Stati dell'UE ed in Italia
- Andamento negli anni dell'indice di felicità nel mondo, in UE ed in Italia
- Analisi dei parametri correlati all'indice di felicità in UE ed in Italia
- Confronto della situazione italiana con la media UE
- Conclusioni


# Data Selections

### Indice di felicità
fonte: [World Happiness Report](https://worldhappiness.report/)  -------  [Download qui](https://www.kaggle.com/datasets/unsdsn/world-happiness?select=2018.csv)

*The World Happiness Report* è un'affermata ricerca a livello internazionale sullo **stato di benessere globale**.

I punteggi e le classifiche di felicità (Happiness Score) utilizzano i dati del sondaggio mondiale Gallup. I punteggi si basano sulle risposte alla principale domanda di valutazione della vita posta nel sondaggio. 
 
Questa domanda chiede agli intervistati di valutare la propria vita facendo riferimento alla scala di Cantril: 0 corrisponde alla peggiore vita possibile e 10 alla condizione migliore.

Le colonne che seguono il punteggio di felicità (HS) stimano la misura in cui HS è influenzato da ciascuno di questi sei fattori: 
- produzione economica, 
- supporto sociale, 
- aspettativa di vita, 
- libertà, 
- assenza di corruzione,
- generosità.

### Indice di libertà
fonte: [The Human Freedom Index](https://www.cato.org/human-freedom-index/2021)  -------  [Download qui](https://www.kaggle.com/datasets/gsutters/the-human-freedom-index)

*The Human Freedom Index* riporta la misura globale delle **libertà personali, civili ed economiche** e viene rilasciato annualmente da *Cato Institute* e da *Fraser Institute*.

L'obiettivo di questo dataset è quello di tracciare un quadro ampio ma ragionevolmente accurato del **grado di libertà generale nel mondo** intesa come assenza di vincoli coercitivi. Utilizza 79 indicatori distinti di libertà personale ed economica, comprende 165 Paesi ed i suoi dati riguardano le analisi dal 2008 al 2021.

### Carico di malattia
fonte: [Global Burden of Disease Collaborative Network.
Global Burden of Disease Study 2019 (GBD 2019) Results.
Seattle, United States: Institute for Health Metrics and Evaluation (IHME), 2020.](http://ghdx.healthdata.org/gbd-results-tool)
rielaborati da Our World in data.  -------  [Download qui](https://ourworldindata.org/grapher/dalys-rate-from-all-causes)

Viene preso come indicatore del carico di malattia della popolazione il DALY (disability-adjusted life year) su 100.000 individui per ogni patologia. Questo  valore indica la misura della **gravità globale di una patologia** espressa come il numero di anni persi a causa della malattia per disabilità o per morte prematura.

### Disoccupazione
fonte: [World Development Indicators - World Bank (2022.05.26)](https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators)
rielaborati da Our World in data.  -------  [Download qui](https://ourworldindata.org/grapher/unemployment-rate)

Questo dataset si riferisce alla quota della forza lavoro che non ha un impiego ma è disponibile e in cerca di un'occupazione espresso in percentuale sulla popolazione nazionale.

### Anni di scolarizzazione previsti
fonte: [UNDP, Human Development Report (2021-22)](https://hdr.undp.org/)
rielaborati da Our World in data.  -------  [Download qui](https://ourworldindata.org/grapher/expected-years-of-schooling)

Gli anni di scolarizzazione previsti sono il numero di anni che un bambino in età scolare dovrebbe trascorrere a scuola e all'università, compresi gli anni di ripetizione. È la somma dei rapporti di iscrizione specifici per età per l'istruzione primaria, secondaria, post-secondaria non terziaria e terziaria.


# Preparation

In [ ]:
# importo librerie necessarie

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import os, shutil
import datetime
from functools import reduce
from math import ceil
import folium
import geojson
import difflib

In [ ]:
# definisco una funzione per estrapolare colori da colormap

def get_colors_from_cmap(cmap, n_colors):
    color_map = cm.get_cmap(cmap, 256)
    colors = color_map(np.linspace(0, 1, n_colors))
    return colors

In [ ]:
# imposto stile e interfaccia grafica

%matplotlib inline
plt.style.use('seaborn-darkgrid')
sns.set_style("darkgrid")
sns.set_palette("viridis")
sns.set_theme("notebook")


In [ ]:
# importo i dataset dalla cartella Data
for f in os.listdir("Data"):
    file, ext = os.path.splitext(f)
    if ext == ".csv":
        globals()[f"{file}_df"] = pd.read_csv(f"Data/{f}")
        print(f"ho creato il dataframe: {file}_df")
    else:
        print(f"non riconosco l'estensione del file {f}, inserire manualmente se necessario")

In [ ]:
# creo cartella immagini se non esiste già

# verifico se la cartella esiste
if not os.path.exists("immagini"):
    os.mkdir("immagini") # creo cartella se non esiste
    print(f"La cartella 'immagini è stata creata.")
else:
    print(f"La cartella immagini esiste già.")


# Data Cleaning & Data Exploration

## Indice Felicità

In [ ]:
WHR_2015_df.sample()

In [ ]:
WHR_2016_df.sample()

In [ ]:
WHR_2017_df.sample()

In [ ]:
WHR_2018_df.sample()

In [ ]:
WHR_2019_df.sample()

### Considerazioni

I dataset del World Happiness Report sono simili ma non identici. I nomi delle colonne variano leggermente ed anche il numero di colonne. E' necessario conformare i nomi delle colonne e scegliere quali utilizzare. 

Mantengo le colonne che sono presenti in tutti gli anni e che hanno dati rilevanti per questa analisi: 

- Country
- Happiness Score
- Economy (GDP per Capita)
- Social Support
- Health (Life Expectancy)
- Freedom to make life choices
- Trust (Government Corruption)
- Generosity
- year


Per rendere più agevole l'analisi unirò i dataset con i dati dei diversi anni in un unico dataset.

**Fasi:**

1. Aggiungo una colonna "year" ad ogni dataframe in cui viene riportato l'anno a cui si riferiscono i dati.
2. Rinomino le colonne di interesse ed elimino quelle non rilevanti
3. Verifico che i nomi degli Stati combacino
4. Unisco i dataset
5. Gestisco i valori mancanti, nulli o 0

#### -1 aggiungo la colonna year

In [ ]:
# 1 aggiungo colonna "year" ai dataframe con anno corrispettivo
for anno in range(2015,2020):
    globals()[f'WHR_{anno}_df']['year'] = anno

#### -2 sistemo le colonne

In [ ]:
# 2 rinomino colonne ed elimino quelle che non servono: 2015
WHR_2015_df.rename(columns={"Family":"Social Support",
                            "Freedom":"Freedom to make life choices"}, 
                   inplace=True)

WHR_2015_df = WHR_2015_df[["year", 
                           "Country",
                           "Happiness Score",
                           "Economy (GDP per Capita)", 
                           "Social Support",
                           "Health (Life Expectancy)",
                           "Freedom to make life choices",
                           "Generosity",
                           "Trust (Government Corruption)"]]

In [ ]:
# 2 rinomino colonne ed elimino quelle che non servono: 2016
WHR_2016_df.rename(columns={"Family":"Social Support",
                            "Freedom":"Freedom to make life choices"}, 
                   inplace=True)

WHR_2016_df = WHR_2016_df[["year", 
                           "Country",
                           "Happiness Score",
                           "Economy (GDP per Capita)", 
                           "Social Support",
                           "Health (Life Expectancy)",
                           "Freedom to make life choices",
                           "Generosity",
                           "Trust (Government Corruption)"]]

In [ ]:
# 2 rinomino colonne ed elimino quelle che non servono: 2017
WHR_2017_df.rename(columns={"Happiness.Score":"Happiness Score",
                            "Economy..GDP.per.Capita.":"Economy (GDP per Capita)",
                            "Health..Life.Expectancy.":"Health (Life Expectancy)",
                            "Family":"Social Support",
                            "Freedom":"Freedom to make life choices",
                            "Trust..Government.Corruption.":"Trust (Government Corruption)"}, 
                   inplace=True)

WHR_2017_df = WHR_2017_df[["year", 
                           "Country",
                           "Happiness Score",
                           "Economy (GDP per Capita)", 
                           "Social Support",
                           "Health (Life Expectancy)",
                           "Freedom to make life choices",
                           "Generosity",
                           "Trust (Government Corruption)"]]

In [ ]:
# 2 rinomino colonne ed elimino quelle che non servono: 2018
WHR_2018_df.rename(columns={"Country or region":"Country",
                            "Score":"Happiness Score",
                            "Social support": "Social Support",
                            "GDP per capita":"Economy (GDP per Capita)",
                            "Healthy life expectancy":"Health (Life Expectancy)",
                            "Perceptions of corruption":"Trust (Government Corruption)"}, 
                   inplace=True)

WHR_2018_df = WHR_2018_df[["year", 
                           "Country",
                           "Happiness Score",
                           "Economy (GDP per Capita)", 
                           "Social Support",
                           "Health (Life Expectancy)",
                           "Freedom to make life choices",
                           "Generosity",
                           "Trust (Government Corruption)"]]

In [ ]:
# 2 rinomino colonne ed elimino quelle che non servono: 2019
WHR_2019_df.rename(columns={"Country or region":"Country",
                            "Score":"Happiness Score",
                            "Social support": "Social Support",
                            "GDP per capita":"Economy (GDP per Capita)",
                            "Healthy life expectancy":"Health (Life Expectancy)",
                            "Perceptions of corruption":"Trust (Government Corruption)"}, 
                   inplace=True)

WHR_2019_df = WHR_2019_df[["year", 
                           "Country",
                           "Happiness Score",
                           "Economy (GDP per Capita)", 
                           "Social Support",
                           "Health (Life Expectancy)",
                           "Freedom to make life choices",
                           "Generosity",
                           "Trust (Government Corruption)"]]

#### -3 verifico nomi Stati

Prima di poter unire i dati in unico dataset bisogna verificare che i nomi degli Stati siano scritti allo stesso modo per ogni anno:

- creo una lista dei nomi che non combaciano tra i vari anni
- correggo i nomi dove possibile
- elimino eventuali Stati non presenti tutti gli anni



In [ ]:
# 3 creo array con nomi degli stati per ogni anno

country_2015 = WHR_2015_df["Country"]
country_2016 = WHR_2016_df["Country"]
country_2017 = WHR_2017_df["Country"]
country_2018 = WHR_2018_df["Country"]
country_2019 = WHR_2019_df["Country"]

# paesi che sono presenti tutti gli anni
evry_year_country = (reduce(np.intersect1d,(country_2015, 
                             country_2016, 
                             country_2017, 
                             country_2018,
                             country_2019)))

print("i Paesi presenti tutti gli anni sono: ", evry_year_country.shape[0])
print("\n")
print("I Paesi che NON sono presenti tutti gli anni sono:")
for anno in range(2015,2020):
    print(anno, "\n", np.setdiff1d(globals()[f"country_{anno}"], evry_year_country))
                                   


##### **correzioni**

Si possono notare alcune differenze facilmente correggibili:
- Hong Kong nel 2017 ha anche la dicitura S.A.R., China
- North Cyprus diventa Northen Cyprus nel 2018
- Taiwan nel 2017 ha anche la dicitura Province of China
- Trinidad and Tobago dal 2018 è Trinidad & Tobago


In [ ]:
# correggo i nomi degli stati

WHR_2017_df.loc[WHR_2017_df["Country"]== "Hong Kong S.A.R., China", "Country"] = "Hong Kong"
WHR_2018_df.loc[WHR_2018_df["Country"]== "Northern Cyprus", "Country"] = "North Cyprus"
WHR_2019_df.loc[WHR_2019_df["Country"]== "Northern Cyprus", "Country"] = "North Cyprus"
WHR_2017_df.loc[WHR_2017_df["Country"]== "Taiwan Province of China", "Country"] = "Taiwan"
WHR_2018_df.loc[WHR_2018_df["Country"]== "Trinidad & Tobago", "Country"] = "Trinidad and Tobago"
WHR_2019_df.loc[WHR_2019_df["Country"]== "Trinidad & Tobago", "Country"] = "Trinidad and Tobago"


In [ ]:
# verifico di aver fatto correttamente le modifiche
evry_year_country = (reduce(np.intersect1d,(country_2015, 
                             country_2016, 
                             country_2017, 
                             country_2018,
                             country_2019)))

print("i Paesi presenti tutti gli anni sono: ", evry_year_country.shape[0])
print("\n")

rispetto ai 141 Paesi precedenti, si sono aggiunti i 4 Paesi che ho modificato.





---

#### -4 unisco i dataframe

Unisco in un unico dataset e seleziono i 145 Paesi

In [ ]:
# 3 unisco i dataframe
WHR_bozza_df = pd.merge(WHR_2015_df, WHR_2016_df, how='outer')
WHR_bozza_df = pd.merge(WHR_bozza_df, WHR_2017_df, how='outer')
WHR_bozza_df = pd.merge(WHR_bozza_df, WHR_2018_df, how='outer')
WHR_bozza_df = pd.merge(WHR_bozza_df, WHR_2019_df, how='outer')


WHR_bozza_df.sample(5)

In [ ]:
# seleziono i 145 Paesi che sono presenti in tutti gli anni
WHR_df = WHR_bozza_df.loc[WHR_bozza_df["Country"] == "PaeseInesistente"] # dataframe vuoto ma con colonne giuste
for country in evry_year_country:
    WHR_df = pd.merge(WHR_df, WHR_bozza_df.loc[WHR_bozza_df["Country"] == country], how="outer") # aggiungo gli Stati presenti ogni anno

WHR_df

il DataFrame ha 725 righe (145 Paesi x 5 anni) quindi l'operazione di merge è andata come previsto

#### -5 valori mancanti

In [ ]:
# verifico dati mancanti
WHR_df.isna().sum()

manca solo il valore di una cella nella colonna Trust

In [ ]:
# verifico presenza di zeri
zero = WHR_df.eq(0).sum()
zero

considerando la natura dei dati, il valore 0 non ha senso come dato,

è più appropriato considerarli dati mancanti paragonabili a NaN.

Sostituisco gli 0 con np.NaN

In [ ]:
# sostituisco gli zeri con np.NaN
WHR_df.replace(0, np.NaN, inplace=True)

In [ ]:
# trovo percentuale di missing values nel dataset
print(f"""
PERCENTUALE DATI MANCANTI:

{(WHR_df.isnull().mean() *100).sort_values(ascending=False)}
""")

**i dati mancanti sono molto pochi, sempre sotto al 1%**

## Indice Libertà

In [ ]:
freedom_df.sample(5)

In [ ]:
freedom_df.info()

### Considerazioni

Il dataset è formato da innumerevoli indici che mettono in relazione diversi aspetti della vita con le libertà personali. Per questa analisi ci interessano gli indici di libertà personale (pf_score), libertà economica (ef_score), libertà umana (hf_score), l'anno ed il nome dello Stato. 

Mantengo le seguenti colonne:

- year
- countries
- hf_score
- pf_score
- ef_score

**fasi:**
1. Seleziono le colonne utili
2. Rinomino le colonne
3. Gestisco i nomi degli Stati
4. Gestisco gli anni
5. Gestisco i valori mancanti, nulli o 0

#### -1 seleziono le colonne

In [ ]:
# verifico i dati delle colonne utili
freedom_df[["year", "countries", "hf_score", "pf_score", "ef_score"]].info()

i dati sembrano consistenti e del tipo corretto

In [ ]:
# prendo solo le colonne con gli indici hf (human freedom), pf (personal freedom) ed ef (economic freedom)
freedom_df = freedom_df[["year", "countries", "hf_score", "pf_score", "ef_score"]]

#### -2 rinomino le colonne

In [ ]:
# cambio il nome alle colonne per standardizzare tutti i dataframe
freedom_df.rename(columns={"countries": "Country",
                           "hf_score": "Human freedom score",
                           "pf_score": "Personal freedom score",
                           "ef_score": "Economic freedom score"}, inplace=True)
freedom_df

#### -3 gestisco gli anni

In [ ]:
freedom_df["year"].min()

In [ ]:
freedom_df["year"].max()

I dati di questo dataset partono dal 2008, visto che i dati riguardanti la felicità partono dal 2015, prendo solo i dati dal 2015 in poi anche per questo dataframe.

In [ ]:
# filtro solo gli anni dopo il 2015
freedom_df = freedom_df.loc[freedom_df["year"] >= 2015,:]
freedom_df

#### -4 gestisco gli Stati


verifico se i nomi degli Stati combacia con il dataset WHR_df

In [ ]:
# verifico se i nomi degli Stati combaciano all'interno del dataframe nei diversi anni

print("numero di Stati:")
for y in range(2015,2020):
    print(f"{y}:", freedom_df.loc[freedom_df["year"]==y,"Country"].count()) # numero Stati per anno


# confronto i nomi degli Stati nei diversi anni
count = 0 
for y in range(2015,2020):
    for y_1 in range(2015,2020):
        if not np.array_equal(freedom_df.loc[freedom_df["year"]==y,"Country"], 
                              freedom_df.loc[freedom_df["year"]==y_1,"Country"]):
            print(f"{y} e {y_1} sono diversi!")
            count += 1
if count == 0:
    print("Sono tutti uguali!")



In [ ]:
# confronto i nomi degli Stati di freedom_df con quelli di WHR_df
C_non_inc = np.setdiff1d(freedom_df.loc[freedom_df["year"]==2015,"Country"], evry_year_country)

print(f"nomi in freedom_df che non sono presenti in WHR_df sono {len(C_non_inc)}")


In [ ]:
# trovo gli Stati spaiati in WHR_df
spaiati = np.setdiff1d(evry_year_country, freedom_df.loc[freedom_df["year"]==2015,"Country"])

print(f"nomi spaiati in WHR_df sono {len(spaiati)}")

In [ ]:
for c in spaiati:
    print(c)

In [ ]:
#cerco somiglianze tra i Paesi che non combaciano
for C in C_non_inc:
    #cerca una corrispondenza simile
    match = difflib.get_close_matches(C, spaiati, n=5, cutoff=0.6)
    if match:
        print(f"Possibile somiglianza tra: {C} e {match}")
        
    # cerco i Paesi che hanno le prime 5 lettere uguali
    for c in spaiati:
        if C[0:5] == c[0:5]: # se hanno le prime 5 lettere uguali
            print(f"Iniziano con le stesse lettere: {C} e {c}")

In [ ]:
# correggo i nomi
freedom_df.loc[freedom_df["Country"]=="Congo, Dem. Rep.", "Country"] = "Congo (Kinshasa)"
freedom_df.loc[freedom_df["Country"]=="Congo, Rep.", "Country"] = "Congo (Brazzaville)"
freedom_df.loc[freedom_df["Country"]=="Egypt, Arab Rep.", "Country"] = "Egypt"
freedom_df.loc[freedom_df["Country"]=="Guinea-Bissau", "Country"] = "Guinea"
freedom_df.loc[freedom_df["Country"]=="Hong Kong SAR, China", "Country"] = "Hong Kong"
freedom_df.loc[freedom_df["Country"]=="Kyrgyz Republic", "Country"] = "Kyrgyzstan"
freedom_df.loc[freedom_df["Country"]=="Russian Federation", "Country"] = "Russia"
freedom_df.loc[freedom_df["Country"]=="Slovak Republic", "Country"] = "Slovakia"
freedom_df.loc[freedom_df["Country"]=="Syrian Arab Republic", "Country"] = "Syria"
freedom_df.loc[freedom_df["Country"]=="Venezuela, RB", "Country"] = "Venezuela"
freedom_df.loc[freedom_df["Country"]=="Yemen, Rep.", "Country"] = "Yemen"


In [ ]:
# verifico la correzione, ho fatto 11 correzioni. dovrei trovarmi 39-11= 28 nomi nella lista dei nomi
# che sono in freedom_df ma non in WHR_df
print("ora i nomi che non combaciano sono:")
len(np.setdiff1d(freedom_df.loc[freedom_df["year"]==2015,"Country"], evry_year_country))



In [ ]:
# mantengo i nomi degli Stati che combaciano

# creo nuovo dataframe vuoto, con solo le colonne
new_freedom_df = freedom_df.loc[freedom_df["Country"]== "PaeseInesistente"] 
for country in evry_year_country:
    new_freedom_df = pd.merge(new_freedom_df, freedom_df.loc[freedom_df["Country"]== country], 
                              how="outer")
    
new_freedom_df

#### -5 valori mancanti

In [ ]:
# percentuali dati mancanti
nan_perc = (new_freedom_df.isnull().mean() *100).sort_values(ascending=False)

# rappresento
print(f"""
PERCENTUALI DATI MANCANTI:

{nan_perc}
""")

In [ ]:
# verifico presenza di zeri
zero = new_freedom_df.eq(0).sum()

# rappresento
print(f"""
ZERI NEI DATI:

{zero}
""")

## Carico Malattia

In [ ]:
burden_disease_df

In [ ]:
burden_disease_df.info()

i dati sono tutti non-nulli e del tipo corretto a parte per la colonna Code.

### Considerazioni

Il dataset contiene i dati dal 1990 al 2019

4 colonne: Country, Code, Year, DALYs

**FASI:**

1. Elimino la colonna Code
2. Rinomino le colonne Year e DALYs
3. Elimino gli anni prima del 2015
4. Gestisco i nomi degli Stati
5. Gestisco i dati mancanti


#### -1 elimino la colonna Code

In [ ]:
# elimino la colonna Code
burden_disease_df.drop(columns=("Code"), inplace=True)

In [ ]:
burden_disease_df.sample(5)

#### -2 rinomino le colonne

In [ ]:
#rinomino colonne
burden_disease_df.rename(columns={"DALYs (Disability-Adjusted Life Years) - All causes - Sex: Both - Age: Age-standardized (Rate)": "Burden disease (DALYs)",
                                  "Year": "year",
                                 "Entity": "Country"},
                           inplace=True)

In [ ]:
burden_disease_df.sample(5)

#### -3 gestisco gli anni

In [ ]:
# elimino gli anni precedenti al 2015
burden_disease_df = burden_disease_df.loc[burden_disease_df["year"]>2014]

In [ ]:
burden_disease_df.head(5)

#### -4 gestisco gli Stati

In [ ]:
# numero degli Stati nel dataset
disease_country = burden_disease_df["Country"].unique()
len(disease_country)

Il numero è così alto perchè oltre agli Stati sono presenti anche diverse regioni territoriali che comprendono diversi Stati, ad esempio: Africa Region, European Region ....

In [ ]:
# verifico gli Stati di disease_country che sono presenti anche in evrevry_year_country
len(np.intersect1d(disease_country, evry_year_country))

137 Paesi sui 145 di WHR_df combaciano

verifico, come per i dataset precedenti, possibili somiglianze di nomi

In [ ]:
# cerco gli 8 Paesi in WHR_df che non hanno un corrispettivo in burden_disease_df 
spaiati = np.setdiff1d(evry_year_country, disease_country)
spaiati

In [ ]:
# trovo gli Stati di burden_disease_df che non sono in WHR_df
C_non_inc = np.setdiff1d(disease_country, evry_year_country)
print(f"gli Stati non inclusi sono {len(C_non_inc)}")

In [ ]:
#cerco somiglianze tra i Paesi che non combaciano
for C in C_non_inc:
    #cerca una corrispondenza simile
    match = difflib.get_close_matches(C, spaiati, n=5, cutoff=0.6)
    if match:
        print(f"Possibile somiglianza tra: {C} e {match}")
        
    # cerco i Paesi che hanno le prime 5 lettere uguali
    for c in spaiati:
        if C[0:5] == c[0:5]: # se hanno le prime 5 lettere uguali
            print(f"Iniziano con le stesse lettere: {C} e {c}")

In [ ]:
# correggo i nomi degli Stati
burden_disease_df.loc[burden_disease_df["Country"]=="Czechia", "Country"] = "Czech Republic"
burden_disease_df.loc[burden_disease_df["Country"]=="Congo", "Country"] = "Congo (Kinshasa)"
burden_disease_df.loc[burden_disease_df["Country"]=="Palestine", "Country"] = "Palestinian Territories"


In [ ]:
# verifico correzioni
disease_country = burden_disease_df["Country"].unique()

new_spaiati = np.setdiff1d(evry_year_country, disease_country)
print(f"ora i Paesi spaiati sono {len(new_spaiati)}")

In [ ]:
# mantengo i nomi degli Stati che combaciano

# creo nuovo dataframe vuoto con solo le colonne
new_burden_disease_df = burden_disease_df.loc[burden_disease_df["Country"]=="PaeseInesistente"] 

for country in evry_year_country:
    new_burden_disease_df = pd.merge(new_burden_disease_df, 
                                     burden_disease_df.loc[burden_disease_df["Country"]== country], 
                                     how="outer")


In [ ]:
new_burden_disease_df

#### -5 valori mancanti

In [ ]:
# percentuali dati mancanti
nan_perc = (new_burden_disease_df.isnull().mean() *100).sort_values(ascending=False)

# rappresento
print(f"""
PERCENTUALI DATI MANCANTI:

{nan_perc}
""")

In [ ]:
# verifico presenza di zeri
zero = new_burden_disease_df.eq(0).sum()

# rappresento
print(f"""
ZERI NEI DATI:

{zero}
""")

## Disoccupazione

In [ ]:
unemployment_df

In [ ]:
unemployment_df.info()

Ad eccezione della colonna Code, le altre colonne sono consistenti e del tipo giusto

### Considerazioni

i dati vanno dal 1991 al 2021

il dataset è composto da 4 features: Entity, Code, Unemployment, total (% of total labor force) (modeled ILO estimate)

**Fasi:**

1. Elimino la colonna Code
2. Rinomino le colonne
3. Gestisco gli anni
4. Gestisco i nomi degli Stati
5. Gestisco i dati mancanti


#### -1 elimino la colonna Code

In [ ]:
# elimino colonne
unemployment_df = unemployment_df.drop(columns=["Code"])

In [ ]:
unemployment_df.sample(5)

#### -2 rinomino le colonne

In [ ]:
# rinomino colonne
unemployment_df = unemployment_df.rename(columns={"Entity": "Country",
                                                  "Year": "year",
                                                  "Unemployment, total (% of total labor force) (modeled ILO estimate)": "Unemployment %"})


In [ ]:
unemployment_df.sample(5)

#### -3 gestisco gli anni

In [ ]:
# mantengo solo gli anni tra il 2015 e il 2019
unemployment_df = unemployment_df.loc[(unemployment_df["year"] > 2014) &  (unemployment_df["year"] < 2020)]

In [ ]:
unemployment_df.head(6)

#### -4 gestisco gli Stati

In [ ]:
# numero degli Stati nel dataset
unemployment_country = unemployment_df["Country"].unique()
len(unemployment_country)

In [ ]:
# verifico gli Stati di unemployment_country che sono presenti anche in evrevry_year_country
len(np.intersect1d(unemployment_country, evry_year_country))

137 Paesi sui 145 di WHR_df combaciano

verifico, come per i dataset precedenti, possibili somiglianze di nomi

In [ ]:
# cerco i 42 Paesi in TOT_df che non hanno un corrispettivo in unemployment_df 
spaiati = np.setdiff1d(evry_year_country, unemployment_country)
print(f"i Paesi spaiati sono {len(spaiati)}")
spaiati

In [ ]:
# trovo gli Stati di unemployment_df che non sono in TOT_df
C_non_inc = np.setdiff1d(unemployment_country, evry_year_country)
print(f"gli Stati non inclusi sono {len(C_non_inc)}")

In [ ]:
#cerco somiglianze tra i Paesi che non combaciano
for C in C_non_inc:
    #cerca una corrispondenza simile
    match = difflib.get_close_matches(C, spaiati, n=5, cutoff=0.6)
    if match:
        print(f"Possibile somiglianza tra: {C} e {match}")
        
    # cerco i Paesi che hanno le prime 5 lettere uguali
    for c in spaiati:
        if C[0:5] == c[0:5]: # se hanno le prime 5 lettere uguali
            print(f"Iniziano con le stesse lettere: {C} e {c}")

In [ ]:
# correggo i nomi degli Stati

unemployment_df.loc[unemployment_df["Country"]=="Democratic Republic of Congo", "Country"] = "Congo (Kinshasa)"
unemployment_df.loc[unemployment_df["Country"]=="Congo", "Country"] = "Congo (Brazzaville)"
unemployment_df.loc[unemployment_df["Country"]=="Palestine", "Country"] = "Palestinian Territories"


In [ ]:
# verifico correzioni
unemployment_country = unemployment_df["Country"].unique()

new_spaiati = np.setdiff1d(evry_year_country, unemployment_country)
print(f"ora i Paesi spaiati sono {len(new_spaiati)}")

In [ ]:
# mantengo i nomi degli Stati che combaciano

# creo nuovo dataframe vuoto, con solo le colonne
new_unemployment_df = unemployment_df.loc[unemployment_df["Country"]=="PaeseInesistente"] 

for country in evry_year_country:
    new_unemployment_df = pd.merge(new_unemployment_df, unemployment_df.loc[unemployment_df["Country"]== country], how="outer")


In [ ]:
new_unemployment_df

#### -5 valori mancanti

In [ ]:
# percentuali dati mancanti
nan_perc = (new_unemployment_df.isnull().mean() *100).sort_values(ascending=False)

# rappresento
print(f"""
PERCENTUALI DATI MANCANTI:

{nan_perc}
""")

In [ ]:
# verifico presenza di zeri
zero = new_unemployment_df.eq(0).sum()

# rappresento
print(f"""
ZERI NEI DATI:

{zero}
""")

## Anni di scolarizzazione previsti

In [ ]:
school_years_df

In [ ]:
school_years_df.info()

Ad eccezione della colonna Code, le altre colonne sono consistenti e del tipo giusto

### Considerazioni

i dati vanno dal 1990 al 2021

il dataset è composto da 4 features: Entity, Code, Expected Years of Schooling

**Fasi:**

1. Elimino la colonna Code
2. Rinomino le colonne
3. Gestisco gli anni
4. Gestisco i nomi degli Stati
5. Gestisco i dati mancanti


#### -1 elimino la colonna Code

In [ ]:
# elimino colonne
school_years_df = school_years_df.drop(columns=["Code"])

In [ ]:
school_years_df.sample(5)

#### -2 rinomino le colonne

In [ ]:
# rinomino colonne
school_years_df = school_years_df.rename(columns={"Entity": "Country",
                                                  "Year": "year"})


In [ ]:
school_years_df.sample(5)

#### -3 gestisco gli anni

In [ ]:
# mantengo solo gli anni tra il 2015 e il 2019
school_years_df = school_years_df.loc[(school_years_df["year"] > 2014) &  (school_years_df["year"] < 2020)]

In [ ]:
school_years_df.head(6)

#### -4 gestisco gli Stati

In [ ]:
# numero degli Stati nel dataset
school_years_country = school_years_df["Country"].unique()
len(school_years_country)

In [ ]:
# verifico gli Stati di school_years_country che sono presenti anche in evrevry_year_country
len(np.intersect1d(school_years_country, evry_year_country))

137 Paesi sui 145 di WHR_df combaciano

verifico, come per i dataset precedenti, possibili somiglianze di nomi

In [ ]:
# cerco i Paesi in TOT_df che non hanno un corrispettivo in school_years_df 
spaiati = np.setdiff1d(evry_year_country, school_years_country)
print(f"i Paesi spaiati sono {len(spaiati)}")
spaiati

In [ ]:
# trovo gli Stati di school_years_df che non sono in TOT_df
C_non_inc = np.setdiff1d(school_years_country, evry_year_country)
print(f"gli Stati non inclusi sono {len(C_non_inc)}")

In [ ]:
#cerco somiglianze tra i Paesi che non combaciano
for C in C_non_inc:
    #cerca una corrispondenza simile
    match = difflib.get_close_matches(C, spaiati, n=5, cutoff=0.6)
    if match:
        print(f"Possibile somiglianza tra: {C} e {match}")
        
    # cerco i Paesi che hanno le prime 5 lettere uguali
    for c in spaiati:
        if C[0:5] == c[0:5]: # se hanno le prime 5 lettere uguali
            print(f"Iniziano con le stesse lettere: {C} e {c}")

In [ ]:
# correggo i nomi degli Stati

school_years_df.loc[school_years_df["Country"]=="Democratic Republic of Congo", "Country"] = "Congo (Kinshasa)"
school_years_df.loc[school_years_df["Country"]=="Congo", "Country"] = "Congo (Brazzaville)"
school_years_df.loc[school_years_df["Country"]=="Czechia", "Country"] = "Czech Republic"
school_years_df.loc[school_years_df["Country"]=="Palestine", "Country"] = "Palestinian Territories"


In [ ]:
# verifico correzioni
school_years_country = school_years_df["Country"].unique()

new_spaiati = np.setdiff1d(evry_year_country, school_years_country)
print(f"ora i Paesi spaiati sono {len(new_spaiati)}")

In [ ]:
# mantengo i nomi degli Stati che combaciano

# creo nuovo dataframe vuoto, con solo le colonne
new_school_years_df = school_years_df.loc[school_years_df["Country"]=="PaeseInesistente"] 

for country in evry_year_country:
    new_school_years_df = pd.merge(new_school_years_df, school_years_df.loc[school_years_df["Country"]== country], how="outer")


In [ ]:
new_school_years_df

#### -5 valori mancanti

In [ ]:
# percentuali dati mancanti
nan_perc = (new_school_years_df.isnull().mean() *100).sort_values(ascending=False)

# rappresento
print(f"""
PERCENTUALI DATI MANCANTI:

{nan_perc}
""")

In [ ]:
# verifico presenza di zeri
zero = new_school_years_df.eq(0).sum()

# rappresento
print(f"""
ZERI NEI DATI:

{zero}
""")

# Data Transformation 

## Mondo

###  Unisco i DataFrame

In [ ]:
# unisco i dataframe

lista_DF = [WHR_df, new_freedom_df, new_burden_disease_df, new_unemployment_df, new_school_years_df] # lista di DF da unire
TOT_df = lista_DF[0]
for df in lista_DF[1:]:
    TOT_df = pd.merge(TOT_df, df, on=["year", "Country"], how='outer')

In [ ]:
TOT_df.sample(5)

### Dati mancanti

**Analizzo il nuovo dataframe comprendente tutti i dati di interesse, ripuliti ed ordinati**

In [ ]:
TOT_df.info()

In [ ]:
# percentuali dati mancanti
nan_perc = (TOT_df.isnull().mean() *100).map(lambda x: round(x,1)).sort_values(ascending=False)

print("Percentuale di dati mancanti nel DataFrame TOT_df:")
nan_perc

In [ ]:
# rappresento

# genero colori per il grafico
colors= get_colors_from_cmap("viridis_r",len(nan_perc))

# genero il grafico
plt.figure(figsize=(10,6))
ax = nan_perc.sort_values().plot(kind="barh", color=colors, width=1)
ax.set_xlim(0,10)
plt.title("percentuale dati mancanti")
ax.bar_label(ax.containers[0], padding=5, fontsize=10)

#salvo immagine
plt.savefig("immagini/percentuale_dati_mancanti.png", bbox_inches="tight")

plt.show()

In [ ]:
# rappresento dati mancanti
fig, ax = plt.subplots(figsize=(10,6), ncols=5, nrows=1, sharey=True)
fig.subplots_adjust(left=0, right=0.9, bottom=0, top=0.85, wspace=0.01)
plt.suptitle("Dati mancanti per anno")

for i in range(0,5):
    sns.heatmap(TOT_df.loc[TOT_df["year"]==(2015+i)].isnull().transpose(), cbar=False, cmap='viridis', xticklabels=False, ax=ax[i])
    ax[i].set_title(2015+i)
    
#salvo immagine
plt.savefig("immagini/dati_mancanti_per_anno.png", bbox_inches="tight")

plt.show()

#### Considerazioni

Le prime 3 features: **year, Country e Happines Score** non hanno dati mancanti, 

i Dtype delle colonne vanno bene, sono adatti al tipo di dato che rappresentano.

Le features con più dati mancanti sono Economic Freedom score e Human Freedom score con il 6,4% di dati mancanti. La percentuale di dati mancanti comunque non è alta e dalla figura "missing data per year" i dati mancanti sembrano essere disposti in modo casuale e senza correlazioni significative.



### Descrizione dati

In [ ]:
TOT_df.sample(5)

In [ ]:
# identifico dimensione DataFrame
TOT_df.shape

In [ ]:
TOT_df.describe()

In [ ]:
# identifico il numero degli Stati presi in esame
TOT_df["Country"].nunique()

Il dataframe ottenuto dopo la pulizia ed il raggruppamento è formato da **730 righe (146 paesi per 5 anni) e da 15 colonne.**

Gli anni presi in considerazione vanno **dal 2015 al 2019.**

il dataframe comprende **145 Stati.**

**Happiness score ha un valore medio di 5.4** tra tutti gli Stati, il valore minimo è 2.8 ed il massimo è 7.7

## Unione Europea

In [ ]:
# creo lista con nomi degli Stati dell'UE
UE_country_list = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"]


In [ ]:
# creo dataframe con Paesi UE

# creo nuovo dataframe vuoto, con solo le colonne
UE_df = TOT_df.loc[TOT_df["Country"]=="PaeseInesistente"] 

# creo il dataframe UE
for country in UE_country_list:
    UE_df = pd.merge(UE_df, TOT_df.loc[TOT_df["Country"]== country], how="outer")

UE_df

il nuovo DataFrame UE_df è formato da 135 colonne (tutti i 27 Paesi europei per 5 anni)

## Italia

In [ ]:
# creo DataFrame con dati dell'Italia
ITA_df = TOT_df.loc[TOT_df["Country"]=="Italy"]
ITA_df

## Andamento HS negli anni

In [ ]:
# creo un DataFrame che rappresenti l'andamento di HS dell'Italia dell'UE e del mondo negli anni.

# raggruppo i df per anno
TOT_y_mean = pd.Series(TOT_df.groupby("year")["Happiness Score"].mean(), name="HS WORLD") 
UE_y_mean = pd.Series(UE_df.groupby("year")["Happiness Score"].mean(), name="HS UE") 
ITA_y_mean = pd.Series(ITA_df.groupby("year")["Happiness Score"].mean(), name="HS ITA") 

# Calcolo la deviazione standard per ogni serie
UE_y_std = pd.Series(UE_df.groupby("year")["Happiness Score"].std(), name="STD UE")
TOT_y_std = pd.Series(TOT_df.groupby("year")["Happiness Score"].std(), name="STD WORLD")

# genero nuovo DF con dati che mi interessano
IUT_y_df = pd.concat([ITA_y_mean, UE_y_mean, UE_y_std, TOT_y_mean, TOT_y_std], axis=1)
# mostro i dati
display(IUT_y_df) 


### Classifiche di Felicità

In [ ]:
# classifica mondiale
WORLD_classifica = TOT_df.groupby("Country")["Happiness Score"].mean().sort_values(ascending=False).round(2).reset_index(drop=False)
WORLD_classifica['index'] = range(1, len(WORLD_classifica) + 1)
WORLD_classifica.set_index('index', inplace=True)

WORLD_classifica

In [ ]:
# classifica UE
UE_classifica = UE_df.groupby("Country")["Happiness Score"].mean().sort_values(ascending=False).round(2).reset_index(drop=False)
UE_classifica['index'] = range(1, len(UE_classifica) + 1)
UE_classifica.set_index('index', inplace=True)

UE_classifica

### Mappa della Felicità

- prendo le coordinate dei confini degli Stati per poter disegnare la mappa.

- aggiusto i nomi degli Stati.

- preparo il file per rappresentare la mappa.

In [ ]:
#converto lista nomi degli Stati in set
country_set = set(evry_year_country)

#carico il file geojson con coordinate dei confini degli Stati
with open("Data/countries.geojson") as f:
    data_stati = geojson.load(f)

# identifico Stati che non combaciano
not_matching_names = set(map(lambda x: x['properties']['ADMIN'], data_stati['features'])) - country_set

#cerco somiglianze tra i Paesi che non combaciano
for C in not_matching_names:
    #cerca una corrispondenza simile
    match = difflib.get_close_matches(C, country_set, n=4, cutoff=0.7)
    if match:
        print(f"Possibile somiglianza tra: {C} e {match}")
        
    # cerco i Paesi che hanno le prime 5 lettere uguali
    for c in country_set:
        if C[0:5] == c[0:5]: # se hanno le prime 5 lettere uguali
            print(f"Iniziano con le stesse lettere: {C} e {c}")

In [ ]:
# Crea un dizionario per i nomi da sostituire
replace_dict = {"United States of America": "United States", 
                "Hong Kong S.A.R.": "Hong Kong", 
                "Northern Cyprus": "North Cyprus", 
                "Dominica": "Dominican Republic", 
                "Palestine": "Palestinian Territories"}

#itero sulle feature del file geojson
for feature in data_stati['features']:
    name = feature['properties']['ADMIN']
    new_name = replace_dict.get(name, name) 
    feature['properties']['ADMIN'] = new_name

#salvo il file modificato
with open("Data/countries_mod.geojson", "w") as f:
    geojson.dump(data_stati, f)
    

# Data Visualization

### Classifica Mondo

In [ ]:
#grafico della classifica di HS nel mondo

#imposto i colori
colors = get_colors_from_cmap("viridis", WORLD_classifica.shape[0])

# imposto grafico
fig, ax = plt.subplots(figsize=(6,35))
plt.title("Classifica Mondo")
plt.xlabel("HS")

# genero grafico e imposto etichette barre
sns.barplot(data=WORLD_classifica, orient="h", x="Happiness Score", y="Country", width=0.9, palette=colors, ax=ax) 
ax.bar_label(ax.containers[0], padding=-30, fontsize=8, color="w") # inserisco valori HS sulle barre

# creo yticks con numero classifica e nome Stato
yticklabels = []
for i, country in enumerate(WORLD_classifica["Country"]):
    yticklabels.append(f"{country}: {i+1}")
ax.set_yticklabels(yticklabels)

# evidenzio l'Italia
italy_index = WORLD_classifica[WORLD_classifica['Country'] == 'Italy'].index[0]-1
ax.containers[0][italy_index].set_edgecolor("red")
ax.containers[0][italy_index].set_linewidth(2)

# aggiungo linea della media
ax.axvline(WORLD_classifica["Happiness Score"].mean(), color="orange", label="Media")

# gestisco legenda
plt.legend(bbox_to_anchor=(1,1))

#salvo immagine
plt.savefig("immagini/classifica_mondo_hs.png", bbox_inches="tight")

plt.show()

Il grafico soprastante mostra la classifica degli Stati del mondo per punteggio Happiness Score.

**I primi Paesi per felicità sono la Danimarca, Norvegia e la Finlandia ed i Paesi Bassi** con rispettivamente un punteggio di 7.55, 7.54 e 7.54.

**I Paesi più infelici del mondo sono il Burundi, la Syria ed il Rwanda** con un punteggio rispettivamente di 3.08, 3,29 e 3.44.

**La media di Happiness Score è di 5.4.**

**L'Italia si trova in 45° posizione con una media di 6.02**, sopra alla media globale.

### Classifica UE

In [ ]:
#grafico della classifica di HS in Unione Europea

#imposto i colori
colors = get_colors_from_cmap("viridis", UE_classifica.shape[0])

# imposto grafico
fig, ax = plt.subplots(figsize=(6,10))
plt.title("Classifica UE")
plt.xlabel("HS")

# creo grafico ed etichette
sns.barplot(data=UE_classifica, orient="h", x="Happiness Score", y="Country", palette=colors, ax=ax) 
ax.bar_label(ax.containers[0], padding=-30, fontsize=10, color="w") # inserisco valori HS sulle barre

# creo yticks con numero classifica e nome stato
yticklabels = []
for i, country in enumerate(UE_classifica["Country"]):
    yticklabels.append(f"{country}: {i+1}")
ax.set_yticklabels(yticklabels)

# evidenzio l'Italia
italy_index = UE_classifica[UE_classifica['Country'] == 'Italy'].index[0]-1
ax.containers[0][italy_index].set_edgecolor("red")
ax.containers[0][italy_index].set_linewidth(2)
# aggiungo line della media
ax.axvline(UE_classifica["Happiness Score"].mean(), color="orange", label="Media")

plt.legend(bbox_to_anchor=(1,1))

#salvo immagine
plt.savefig("immagini/classifica_UE_hs.png", bbox_inches="tight")

plt.show()

Il grafico soprastante mostra la classifica degli Stati UE per punteggio Happiness Score.

**I primi Paesi per felicità sono la Danimarca, la Finlandia ed i Paesi Bassi** con rispettivamente un punteggio di 7.55, 7.54 e 7.40.

**I Paesi più infelici dell'Unione Europea sono la Bulgaria, la Grecia ed il Portogallo** con un punteggio rispettivamente di 4.62, 5.15 e 5.3.

**La media di Happiness Score in Unione Europea è di 6.27.**

**L'Italia si trova in 15° posizione con una media di 6.02**

### Mappa della Felicità

#### Mondo

In [ ]:
# genero la mappa Mondo
mappaMondo = folium.Map(location=[43,12], 
                   zoom_start=1.5, 
                   tiles="Stamen Watercolor", 
                   width='100%', 
                   height='80%',
                   no_touch=True,
                   zoom_control=False)

# aggiungo i confini
folium.GeoJson("Data/countries_mod.geojson").add_to(mappaMondo)

# coloro in base a Happiness Score
folium.Choropleth(
    geo_data=data_stati,
    data=WORLD_classifica,
    columns=("Country", "Happiness Score"),
    key_on="feature.properties.ADMIN",
    bins=20,
    fill_color="viridis",
    nan_fill_color='black',
    fill_opacity=0.8,
    line_color='black',
    line_weight=1,
    line_opacity=1,
    legend_name='Happiness Score').add_to(mappaMondo)

#salvo immagine
mappaMondo.save("immagini/mappa_mondo_hs.html")

mappaMondo


La mappa mostra il **valore di HS nei diversi Stati del mondo** (media dei 5 anni presi in considerazione).

Si può notare come **i Paesi del nord Europa e del nord America siano i più felici insieme all'Australia.**

**I Paesi con indice di felicità più basso invece sono distribuiti principalmente nel continente africano e nel sud-ovest asiatico.**

#### Unione Europea

In [ ]:
# genero la mappa UE
mappaUE = folium.Map(location=[55,15], 
                   zoom_start=3.4, 
                   tiles="Stamen Watercolor", 
                   width='100%', 
                   height='80%',
                   no_touch=True,
                   zoom_control=False)

# aggiungo i confini
folium.GeoJson("Data/countries_mod.geojson").add_to(mappaUE)

# coloro in base a Happiness Score
folium.Choropleth(
    geo_data=data_stati,
    data=UE_classifica,
    columns=("Country", "Happiness Score"),
    key_on="feature.properties.ADMIN",
    bins=20,
    fill_color="viridis",
    nan_fill_color='black',
    fill_opacity=0.8,
    line_color='black',
    line_weight=1,
    line_opacity=1,
    legend_name='Happiness Score').add_to(mappaUE)

#salvo immagine
mappaUE.save("immagini/mappa_UE_hs.html")

mappaUE

Per quanto riguarda l'**Unione Europea** si può osservare che **i Paesi con HS più alto sono quelli del nord (Danimarca, Svezia, Finlandia...)** mentre **i più infelici quelli del sud-est (Grecia, Ungheria, Bulgaria) ed il Portogallo.**



### Distribuzione HS

In [ ]:
# metto a confronto le distribuzioni ed i valori di HS
plt.figure(figsize=(10,6))
plt.title("distribuzione di Happiness Score, confronto tra Mondo, Unione Europea ed Italia")
colors = get_colors_from_cmap("viridis",3) #identifico i colori del grafico

# distribuzione e media globale
TOT_media = np.round(TOT_df["Happiness Score"].mean(),2)
sns.histplot(data=TOT_df["Happiness Score"], kde=True, color=colors[0])
plt.axvline(TOT_media, label=f'World, media = {TOT_media}', color=colors[0], linewidth=2)

# distribuzione e media europea
UE_media = np.round(UE_df["Happiness Score"].mean(),2)
sns.histplot(data=UE_df["Happiness Score"], kde=True, color=colors[1])
plt.axvline(UE_media, label=f'UE, media = {UE_media}', color=colors[1], linewidth=2)

# media italiana
ITA_media = np.round(ITA_df["Happiness Score"].mean(),2)
plt.axvline(ITA_media, label=f'ITA, media = {ITA_media}', color=colors[2], linewidth=2)

# creo la legenda e modifico settaggi
legend = plt.legend(loc=2)
for line in legend.get_lines():
    line.set_linewidth(5)

#salvo immagine
plt.savefig("immagini/distribuzione_hs.png", bbox_inches="tight")
    
plt.show()

Dal grafico delle distribuzioni si nota come i **Paesi dell'UE abbiano mediamente un valore di HS più elevato rispetto al resto del mondo**. I punteggi di HS in UE infatti vanno da 4.5 a 7.5 circa con media 6.27 rispetto ai dati di HS globali che partano da un valore di circa 2.5 con una media di 5.4.

Si nota inoltre che il valore medio di **HS in Italia è di 6.02, maggiore della media globale ma minore della media europea. **


### Andamento HS negli anni

In [ ]:
#genero il grafico
colors = get_colors_from_cmap("viridis", 3) # imposto i colori
fig, ax = plt.subplots(figsize=(10,6))

# Disegnare andamento HS nel tempo e deviazione std
ax.errorbar(TOT_y_mean.index+0.02, TOT_y_mean.values, yerr=TOT_y_std.values, 
            fmt='->', color=colors[0], markersize=10, linewidth=4, ecolor='gray', 
            elinewidth=1, capsize=5, label="WORLD") #mondo
ax.errorbar(UE_y_mean.index-0.02, UE_y_mean.values, yerr=UE_y_std.values,  
            fmt='->', color=colors[1], markersize=10, linewidth=4, ecolor='k', 
            elinewidth=1, capsize=10, label="UE") #UE
ax.plot(ITA_y_mean, color=colors[2], linewidth=4, marker=">", markersize=10, 
        label="ITA") #ITA
ax.set_xticks([2015,2016,2017,2018,2019])

plt.legend(bbox_to_anchor=(1,1),handleheight=2)

#salvo immagine
plt.savefig("immagini/andamento_negli_anni_hs.png", bbox_inches="tight")

plt.show()


Si può osservare un **andamento crescente di tutte e tre le medie di HS**. 
La media globale incrementa di poco il proprio valore negli anni, solo 0.1 punti.
L'Unione Europea e l'Italia incrementano leggermente di più, aumentando di 0.3 punti in 5 anni.

Si osserva inoltre che **la media di HS dell'UE rimane la più alta delle tre per tutti i 5 anni**, seguita dall'Italia ed infine dal mondo.

Si osserva che la media di **HS in Italia è sempre più bassa della media Europea ma più alta della media globale.

Rispetto al contesto socio-economico in cui il nostro Paese vive ed agisce (UE) l'Italia risulta avere un indice HS più basso.

Proviamo a capire se ci sono parametri correlati ad HS che l'Italia potrebbe migliorare.

### Correlazioni con HS

In [ ]:
# cerco correlazioni tra le colonne e HS per identificare le features più importanti su cui poter agire

fig, axs = plt.subplots(1, 2, figsize=(6, 5), sharey=True)
fig.suptitle("Correlazioni con HS")
axs[0].set_title("World")
axs[1].set_title("UE")

# Calcolo la matrice di correlazione per ogni DataFrame
TOT_corr = TOT_df.corr(numeric_only=True)
UE_corr = UE_df.corr(numeric_only=True)

# Seleziono la colonna 'Happiness Score'
score_corr_TOT = TOT_corr['Happiness Score']
score_corr_UE = UE_corr['Happiness Score']

# Disegno il grafico di heatmap per ogni DataFrame
sns.heatmap(score_corr_TOT.to_frame(), annot=True, cmap="viridis", ax=axs[0])
sns.heatmap(score_corr_UE.to_frame(), annot=True, cmap="viridis", ax=axs[1])

#salvo immagine
plt.savefig("immagini/correlazioni_mondo-UE_hs.png", bbox_inches="tight")

plt.show()


In [ ]:
# grafico solo UE ordinato per correlazione

# ordino
sorted_score_corr_UE = score_corr_UE.sort_values(ascending=False)
# imposto grafico
fig, ax = plt.subplots(figsize=(4,6))
plt.title("Correlazioni con Happiness Score nei Paesi UE\n", fontdict={"fontsize":15})
sns.heatmap(sorted_score_corr_UE.to_frame(), annot=True, cmap="viridis",linewidths=1, linecolor='k', ax=ax)

#salvo immagine
plt.savefig("immagini/correlazioni_UE_sorted.png", bbox_inches="tight")

plt.show()

I valori ed i colori nella tabella soprastante rappresentano il **grado di correlazione tra le varie colonne del DataFrame e HS.**
**Valori negativi** indicano una **correlazione inversa** con grado che aumenta da 0 a -1.
**Valori positivi** indicano invece una **correlazione di tipo diretto** con grado che aumenta da 0 a 1.

I grafici riguardanti il mondo, a sinistra, e gli Stati dell'Unione Europea, a destra, sono affiancati.

Le due heatmap sono simili ma non uguali, sono infatti presenti delle differenze importanti per questa analisi.

**Queste differenze fanno capire che in base alla zona del mondo, alla situazione socio-economica e culturale, cambia il modo in cui un determinato parametro influenza il benessere dei cittadini.**

Di conseguenza risulta fondamentale capire il contesto socio-politico-culturale del Paese per poter attuare azioni politiche mirate all'incremento della felicità dei cittadini.

**Per questo motivo confrontare i dati dell'Italia con quelli dell'Unione Europea, invece che con quelli mondiali, risulta più appropriato in quanto il contesto sociale, politico ed economico è più simile.**

Analizzando il grafico riguardante l'Unione Europea si osserva che:
- Le uniche colonne **correlate negativamente con HS sono Burden disease(DALYs) e Unemployment%, rispettivmente con valori di -0.6 e -0.33.**
- la colonna con **indice di correlazione più alto è Trust (Government Corruption)** che indica il grado di fiducia nelle istituzioni da parte dei cittadini, con un valore di **0.84**
- la colonna con la minore correlazione è **year**, che sta ad indicare un **lieve incremento di HS negli anni** ma non molto significativo
- **tutte le altre colonne sono correlate positivamente con un indice tra lo 0.43 e lo 0.73**


### Parametri correlati ad HS  --> ITA vs UE

In [ ]:
#imposto i colori
colors = get_colors_from_cmap("viridis", 3)

# Calcolo media e std 
mean_df = UE_df.groupby('Country').mean(numeric_only=True) # media per ogni stato
std_df = UE_df.groupby('Country').std(numeric_only=True) # std per ogni stato
mean_all = UE_df.mean(numeric_only=True) # media totale
std_all = UE_df.std(numeric_only=True) # std totale

# imposto il grafico
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(10, 10))
axs=axs.ravel()

#itero attraverso le colonne del DataFrame per creare i diversi assi
for idx, column in enumerate(UE_df.columns[3:]):
    axs[idx].bar(mean_df.loc['Italy'].name, mean_df.loc['Italy'][column], 
                 color=colors[2], label="ITA") # barra ITA
    axs[idx].bar("mean", mean_all[column], color=colors[1], label="Media UE") # barra UE
    axs[idx].errorbar("mean", mean_all[column], std_all[column], 
                      fmt='', capsize=150, color='k', label="dev.std") # error bar
    axs[idx].set_title(column)
    axs[idx].set_ylim(mean_all[column]-(1.5*std_all[column]),
                      mean_all[column]+(1.5*std_all[column])) # regolo ylim in base a std

plt.suptitle("Parametri correlati ad HS,  ITA vs UE", horizontalalignment="right")
plt.tight_layout()

# imposto la legenda
dev_std_handle = axs[idx].errorbar("mean", mean_all[column], std_all[column],
                      fmt='', capsize=5, color='k', label="dev.std") # handle dev std
ITA_handle = axs[idx].bar(mean_df.loc['Italy'].name, mean_df.loc['Italy'][column], 
                 color=colors[2], label="ITA") # handle ITA
UE_handle = axs[idx].bar("mean", mean_all[column], color=colors[1], label="Media UE") # handle UE

plt.legend(handles=[ITA_handle, UE_handle, dev_std_handle], bbox_to_anchor=(1, 3.8),ncols=4, frameon=True)

#salvo immagine
plt.savefig("immagini/parametri_correlati_ITA_vs_UE.png", bbox_inches="tight")

plt.show()

I grafici soprastanti mettono in evidenza la differenza tra i valori correlati ad HS dell'Italia e della media in UE.

Le differenze più significative riguardano: la libertà di fare scelte di vita ed il carico di malattia, che hanno una differenza maggiore ad una deviazione standard. 

**La libertà di fare scelte di vita** , che ha una correlazione con HS di **0.73, risulta minore in Italia rispetto alla media UE. Questo indica come incida fortemente sulla felicità della popolazione ed è un aspetto su cui focalizzare l'attenzione.**

**Il carico di malattia**, che ha un indice di correlazione negativo con HS, pari a **-0.6, è minore in Italia rispetto alla media UE, ad indicare la migliore situazione sanitaria italiana. Questo è confermato anche dal valore dell'aspettativa di vita che, anche se in minor misura, indica una situazione migliore in Italia.**

**La fiducia nelle istituzioni** (Trust), che ha l'indice di correlazione con HS più alto, pari a **0.84, risulta minore in Italia quasi di una deviazione standard. Questo indica che la fiducia nelle istituzioni è un aspetto cruciale su cui l'Italia dovrebbe lavorare per aumentare la felicità dei cittadini.**

**La disoccupzione**, con indice di correlazione **-0.33, risulta maggiore in Italia indicando una situazione peggiore nel nostro Paese rispetto alla media UE.** Tuttavia la differenza rispetto alla media UE non è elevata e l'indice di correlazione non sembra molto rilevante. Questo indica che la disoccupazione è un aspetto su cui l'Italia potrebbe lavorare per incrementare l'Happiness Score italiano ma non è un aspetto cruciale come i parametri visti precedentemente.

Per gli altri parametri la differenza tra Italia ed UE è irrisoria indicando che la situazione italiana in questi ambiti è in linea con quella europea.

### Andamento parametri correlati ad HS

In [ ]:
# seleziono colori grafico
colors=get_colors_from_cmap("viridis", 3) 

fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(10,10), sharex=True)
axs=axs.ravel()

for i, column in enumerate(UE_df.columns[3:]):
    
    UEf_grouped = UE_df.groupby("year")[column].mean() # raggruppo per anno
    UEf_std = UE_df.groupby("year")[column].std() # prendo dev std UE
    ITAf_grouped = ITA_df.groupby("year")[column].mean() # raggruppo per anno
    # grafico UE
    axs[i].errorbar(UEf_grouped.index, UEf_grouped.values, 
                            yerr=UEf_std.values,  fmt='->', color=colors[1], 
                            markersize=7, linewidth=2, ecolor='black', 
                            elinewidth=1, capsize=3, label="UE")
    # grafico ITA
    axs[i].plot(ITAf_grouped, color=colors[2], linewidth=2, marker=">", label="ITA")
    axs[i].set_title(column) # titoli assi
    axs[i].set_xticks=[UEf_grouped.index]


fig.suptitle("Andamento parametri nel tempo: ITA vs UE")
plt.tight_layout()
legend = plt.legend(ncol=2, bbox_to_anchor=(-0.5, 5.25), frameon=True)

#salvo immagine
plt.savefig("andamento_parametri_nel_tempo_ITA_vs_UE.png", bbox_inches="tight")

plt.show()


i grafici soprastanti mostrano l'andamento negli anni dei diversi parametri rappresentati dalle colonne del DataFrame dell'Italia e dell'Unione Europea.

si può osservare che:
- Economy (GDP per capita), Social Support, Human Freedom Score, Personal Freedom Score e Generosity hanno valori simili ed un andamento negli anni comparabile.

- La colonna con la correlazione più alta con HS, **Trust (Government corruption) (correlazione con HS 0.84) ha un andamento simile tra Italia e UE ma con valori sempre inferiori in Italia. La differenza è di quasi una deviazione standard. Questi dati indicano che in Italia la fiducia nelle istituzioni è poca e questo incide molto sulla felicità della popolazione.**

- La seconda colonna per correlazione con HS è **Freedom to make life choices, con valore di 0.73**. Nel grafico soprastante si può notare come l'**Italia abbia valori** di Freedom to make life choices **costantemente inferiori alla media UE di oltre una dev. std. Questo indica che la libertà di fare scelte di vita è un aspetto cruciale su cui focalizzare l'attenzione per poter aumentare il benessere dei cittadini.**

- **Dal punto di vista sanitario l'Italia sembra invece trovarsi in una situazione migliore rispetto alla media dell'UE. Sia i dati sull'aspettativa di vita che quelli sul carico di malattia sono migliori rispetto alla media europea.** L'aspettativa di vita infatti è sempre maggiore della media europea mentre il carico di malattia è inferiore alla media UE in tutti gli anni, entrambi con una differenza pari o poco superiore ad una dev.std.

- I dati riguardanti gli **anni previsti di scolarizzazione indicano un andamento similare tra Italia e UE ma con valori leggermente inferiori in Italia.** La differenza è minore di metà della dev. std.

- Per quanto riguarda la **libertà economica troviamo una situazione simile alla precedente ma con una leggera tendenza al peggioramento in Italia.**

- **La percentuale di disoccupazione infine indica un miglioramento graduale negli anni sia in Italia che in UE ma con valori di disoccupazione sempre minori in UE e un miglioramento meno deciso in Italia.** la differenza aumenta negli anni fino a superare una dev. std. nel 2019.


# Conclusioni

L'analisi condotta permette di fare le seguenti considerazioni:

- **L'indice di felicità in Italia (6.0) risulta maggiore rispetto alla media globale (5.4) ma è minore rispetto alla media UE (6.3).**

- L'Italia si trova al **45**° posto nella classifica globale per HS e al **15°** in quella europea.

- **L'indice di felicità è cresciuto leggermente in tutto il mondo dal 2015 al 2019, indicando probabimente un complessivo miglioramento delle condizioni di vita**

- I parametri correlati maggiormente con HS in UE sono: 
    - La fiducia nelle istituzioni,
    - La libertà di fare scelte di vita,
    - Il guadagno economico personale


- **Situazione italiana:**
    - **Punti di forza:**
        - **Situazione sanitaria:** l'Italia è abbondantemente sopra alla media UE per quanto riguarda il carico di malattia e leggermente sopra alla media per quanto riguarda l'aspettativa di vita. Questo indica una situazione positiva in ambito sanitario che influisce notevolmente sulla felicità degli italiani.
    - **Punti deboli:**
        - **Libertà di fare scelte di vita:** i dati riguardanti questo aspetto mostrano una differenza consistente a sfavore dell'Italia. Questo parametro inoltre ha una forte correlazione con l'indice di felicità. Questo indica che misure volte a migliorare la libertà delle persone di poter fare scelte di vita sono cruciali per migliorare la felicità degli italiani.
        - **Fiducia nel governo:** i dati riguardanti questo aspetto mostrano una differenza significativa tra l'Italia e la media UE. Questo parametro è quello maggiormente correlato con l'indice di felicità e perciò migliorare la fiducia dei cittadini nelle istituzioni è vitale per poter aumentare l'HS italiano.
        - **Disoccupazione:** anche se in misura inferiore rispetto ai due parametri precedenti, la situazione italiana è peggiore rispetto alla media UE e questo incide sull'HS dell'Italia. Risulta quindi importante focalizzare l'attenzione anche su questo aspetto.